In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

from gensim.models import KeyedVectors

from keras import Model
from keras.layers import Input, LSTM, Dense, Embedding



In [2]:
seq_length = 250
step = seq_length
num_words = 10000
# maxlen = 1000

df = pd.read_csv("data.csv", index_col="index")
df = df.reindex(np.random.permutation(df.index))
df.columns = ["label", "text"]
labels = df["label"].to_numpy()
texts = df["text"].to_numpy()

In [3]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(texts)
sequences = np.array(tokenizer.texts_to_sequences(texts))
# x = pad_sequences(x, maxlen=maxlen, padding="post")

label_encoder = LabelEncoder()
label_encoder.fit(["Rock", "Pop", "Hip Hop"])
encoded_labels = label_encoder.transform(labels)

x = []
y = []
for label_ind, sequence in enumerate(sequences):
    for i in range(0, len(sequence) - seq_length, step):
        x.append(sequence[i: i+seq_length])
        y.append(encoded_labels[label_ind])

# x = to_categorical(x)
x = np.array(x)
y = to_categorical(y)

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

print(train_x.shape)
print(train_y.shape)


(59717, 250)
(59717, 3)


In [4]:
n_lstm_layers = 6

input_layer = Input(x[0].shape)
z = input_layer
z = Embedding(num_words, 300)(z)
for i in range(n_lstm_layers):
    z = LSTM(128, return_sequences=i != n_lstm_layers-1)(z)
out = Dense(3, activation="softmax")(z)
model = Model(inputs=[input_layer], outputs=[out])
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 250, 300)          3000000   
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 250, 128)          220160    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 250, 128)          132096    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 250, 128)          132096    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 250, 128)          132096    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 250, 128)         

In [5]:
model.fit(train_x, train_y, batch_size=32, validation_data=(test_x, test_y), epochs=10)

Epoch 1/10
1867/1867 [==============================] - 213s 114ms/step - loss: 0.9493 - accuracy: 0.5285 - val_loss: 0.9187 - val_accuracy: 0.5633
Epoch 2/10
1867/1867 [==============================] - 219s 118ms/step - loss: 0.8248 - accuracy: 0.5936 - val_loss: 0.8003 - val_accuracy: 0.6062
Epoch 3/10
1867/1867 [==============================] - 206s 111ms/step - loss: 0.7712 - accuracy: 0.6342 - val_loss: 0.8003 - val_accuracy: 0.6223
Epoch 4/10
1867/1867 [==============================] - 219s 117ms/step - loss: 0.7441 - accuracy: 0.6535 - val_loss: 0.7687 - val_accuracy: 0.6216
Epoch 5/10
 784/1867 [===========>..................] - ETA: 2:06 - loss: 0.7158 - accuracy: 0.6728